### Deformation Field Overlaying into the Moving Image

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from   natsort import natsorted
from   scipy.ndimage import gaussian_filter, map_coordinates

In [2]:
# Function to get the deformation field and moving images
def get_file_paths(directory):
    file_types = {
        'img1.nii.gz'   : [],
        'img2.nii.gz'   : [],
        'seg2.nii.gz'   : [],
        'w_img.nii.gz'  : [],
        'w_seg.nii.gz'  : [],
        'flow_4d.nii.gz': [],
        
    }
    
    for filename in os.listdir(directory):
        for file_suffix in file_types:
            if filename.endswith(file_suffix):
                file_types[file_suffix].append(os.path.join(directory, filename))
                break
    for key in file_types:
        file_types[key] = natsorted(file_types[key])
    
    return file_types

In [3]:
def load_nifti_image(file_path):
    nii   = nib.load(file_path)
    image = nii.get_fdata()
    return image, nii.affine

In [4]:
def load_deformation_field(file_path):
    nii = nib.load(file_path)
    deformation_field = nii.get_fdata()
    return deformation_field

In [ ]:
def get_ct_views(ct_scan):
     # ct_scan: shape (192, 160, 256) => (z, y, x)
    
    # Axial view: Slices along the z-axis, output shape (160, 256) per slice
    axial_slices = [ct_scan[i, :, :] for i in range(ct_scan.shape[0])]
    
    # Sagittal view: Slices along the x-axis, output shape (192, 160) per slice
    sagittal_slices = [ct_scan[:, :, i] for i in range(ct_scan.shape[2])]
    
    # Coronal view: Slices along the y-axis, output shape (192, 256) per slice
    coronal_slices = [ct_scan[:, i, :] for i in range(ct_scan.shape[1])]
    
    return axial_slices, sagittal_slices, coronal_slices

In [ ]:
def get_deformation_field_views(deformation_field):
    # deformation_field: shape (192, 160, 256, 3)
    
    # Axial view: Take slices along the z-axis (displacement in x and y directions)
    axial_deformation = [deformation_field[i, :, :, :2] for i in range(deformation_field.shape[0])]  # (160, 256, 2) per slice
    
    # Sagittal view: Take slices along the x-axis (displacement in y and z directions)
    sagittal_deformation = [deformation_field[:, :, i, 1:] for i in range(deformation_field.shape[2])]  # (192, 160, 2) per slice
    
    # Coronal view: Take slices along the y-axis (displacement in x and z directions)
    coronal_deformation = [deformation_field[:, i, :, ::2] for i in range(deformation_field.shape[1])]  # (192, 256, 2) per slice
    
    return axial_deformation, sagittal_deformation, coronal_deformation

In [5]:
def draw_grid(image, grid_spacing=10, grid_value=1):
    grid_image = np.zeros(image.shape)
    # Drawing the vertical and horizontal lines
    grid_image[:, ::grid_spacing] = grid_value
    grid_image[::grid_spacing, :] = grid_value
    return grid_image

In [7]:
def blur_grid(grid_image, sigma=0.25):
    blurred_grid = gaussian_filter(grid_image, sigma=sigma)
    return blurred_grid

In [8]:
def add_grid_to_image(image, grid_image):
    modified_image = image + grid_image
    return modified_image

In [9]:
def apply_deformation(image, deformation_field):
    
    # Create a meshgrid of coordinates
    coords = np.meshgrid(np.arange(image.shape[0]), 
                         np.arange(image.shape[1]), 
                         indexing='ij')
    coords = np.array(coords)
   
    
    # Apply the deformation field to the coordinates
    deformed_coords_0 = coords[0] + deformation_field[..., 0]
    deformed_coords_1 = coords[1] + deformation_field[..., 1]
    
    # Interpolate the image at the new coordinates
    warped_image = map_coordinates(image, [deformed_coords_0, deformed_coords_1], order=1)
    return warped_image

In [10]:
# Function to visualize slices
def visualize_slices(data, title="Image Slices", slices=None):
    """
    Visualizes slices of a 3D image.

    Parameters:
    - data: 3D numpy array representing the image data.
    - title: Title for the plot.
    - slices: Tuple of three integers representing the indices of the slices 
              to visualize in the axial, coronal, and sagittal planes. 
              If None, the middle slices are used.
    """
    #if slices is None:
    #    slices = (data.shape[0] // 2, data.shape[1] // 2, data.shape[2] // 2)
    slice_x = data.shape[0]
    slice_y = data.shape[1]


    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(title)

    # Axial slice (viewed from the top of the head)
    axes[0].imshow(data[slices[0], :, :].T, cmap='gray', origin='lower')
    axes[0].set_title(f'Sagital slice {slices[0]}')
    
    # Coronal slice (viewed from the front)
    axes[1].imshow(data[:, slices[1], :].T, cmap='gray', origin='lower')
    axes[1].set_title(f'Coronal slice {slices[1]}')

    # Sagittal slice (viewed from the side)
    axes[2].imshow(data[:, :, slices[2]].T, cmap='gray', origin='lower')
    axes[2].set_title(f'Axial slice {slices[2]}')
    plt.show()

In [11]:
def main(image_path, seg_path, warp_img_path, warp_seg_path, deformation_path, output_path, grid_spacing=10, sigma=0.5):
    # Step1: Loading the moving image and the deformation field in 4d
    image, affine    = load_nifti_image(image_path)
    segm,  affine    = load_nifti_image(seg_path)
    warpped_image, _ = load_nifti_image(warp_img_path)
    warpped_segm,  _ = load_nifti_image(warp_seg_path)
    def_field_4d     = load_deformation_field(deformation_path)
    print('image: ', image.shape)
    print('segm: ', segm.shape)
    print('warped img: ', warpped_image.shape)
    print('warped segm: ', warpped_segm.shape)
    print('def_field_4d: ', def_field_4d.shape)
    
    # Step2: Drawing and bluring the grid
    grid_ax, grid_sa, grid_co = draw_grid_on_views(image, grid_spacing=grid_spacing)
    blur_ax = blur_grid(grid_ax, sigma=sigma)
    blur_sa = blur_grid(grid_sa, sigma=sigma)
    blur_co = blur_grid(grid_co, sigma=sigma)
    print(grid_ax.shape)
    print(grid_sa.shape)
    print(grid_co.shape)
    print(blur_co.shape)

    # Step3: Adding the grid to the moving image
    '''modified_image = add_grid_to_image(image, blurred_grid)
    modified_segm  = add_grid_to_image(segm, blurred_grid)
    
    # Step4: Applying the deformation field to the moving image
    warped_image = apply_deformation(modified_image, def_field_4d)
    warped_segm  = apply_deformation(modified_segm, def_field_4d)
    
    # Step5: Visualizing moving image outputs
    visualize_slices(image, title='Moving Image')
    visualize_slices(blurred_grid, title='Blurred Image')
    visualize_slices(modified_image, title='Moving Image with Blurred Grid')
    visualize_slices(warped_image,   title='Deformed Moving Image with Blurred Grid')
    visualize_slices(warpped_image,   title='Deformed Moving Image - Output Model')
    
    # Step6: Visualizing moving segmentation outputs
    visualize_slices(segm, title='Moving Segmenation')
    visualize_slices(modified_segm, title='Moving Segmentation with Blurred Grid')
    visualize_slices(warped_segm,   title='Deformed Moving Segmentation with Blurred Grid')
    visualize_slices(warpped_segm,   title='Deformed Moving Segmentation - Output Model')'''
    
      

##### VoxelMorph

In [12]:
# VoxelMorph Directory
vxm_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/Jul31-053533_Abtrain_VXMx1___/'
vxm_defGrid_dir  = vxm_directory + 'defGrid_figures/'
image_paths   = get_file_paths(vxm_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]

In [13]:
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')


image:  (192, 160, 256)
segm:  (192, 160, 256)
warped img:  (192, 160, 256)
warped segm:  (192, 160, 256)
def_field_4d:  (192, 160, 256, 3)
(160, 256)
(192, 160)
(192, 256)
(192, 256)


In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

##### VTN

In [ ]:
# VTN Directory
vtn_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/Aug06-211340_Abtrain_VTNx3___/'
vtn_defGrid_dir  = vtn_directory + 'defGrid_figures/'
image_paths   = get_file_paths(vtn_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]

main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')


In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

##### TSM

In [ ]:
# TSM Directory
tsm_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/Aug07-042242_Abtrain_TSMx1___/'
tsm_defGrid_dir  = tsm_directory + 'defGrid_figures/'
image_paths   = get_file_paths(tsm_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')

In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

#### ALN V1

In [ ]:

aln_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/Aug12-033920_Abtrain_ALNx1___/'
aln_defGrid_dir  = aln_directory + 'defGrid_figures/'
image_paths   = get_file_paths(aln_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')

In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

#### ALN V2

In [ ]:
aln_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/Aug29-191908_Abtrain_ALNx3___/'
aln_defGrid_dir  = aln_directory + 'defGrid_figures/'
image_paths   = get_file_paths(aln_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')

In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

#### Elastix

In [32]:
def load_nifti_image(file_path):
    nii   = nib.load(file_path)
    image = nii.get_fdata()
    return image, nii.affine

In [42]:
import SimpleITK as sitk
elx_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/elastix_Abd/'
elx_defGrid_dir  = elx_directory + 'defGrid_figures/'
image_paths   = get_file_paths(elx_directory)
img1_paths    = image_paths['img1.nii.gz']
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img1_0_paths    = img1_paths[0]
img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img1_0 = sitk.ReadImage(img1_0_paths) #load_nifti_image(img1_0_paths)
img2_0 = sitk.ReadImage(img2_0_paths) #load_nifti_image(img1_0_paths)
print(img1_0.GetSize())
#img2_0,_ = load_nifti_image(img2_0_paths)

#im_fixed = sitk.GetImageFromArray(img1_0) # 1 x 1 x 192 x 192 x 208 -----> 192 x 192 x 208 -----> 208 x 192 x 192
#im_moving = sitk.GetImageFromArray(img2_0)

# 
#elastixImageFilter = sitk.ElastixImageFilter()
#elastixImageFilter.SetFixedImage(img1_0)
#elastixImageFilter.SetMovingImage(img2_0)

# Use default parameter map (elastix configuration for non-rigid registration)
parameterMapVector = sitk.VectorOfParameterMap()
defaultParameterMap = sitk.GetDefaultParameterMap("bspline")
#print(defaultParameterMap)
sitk.PrintParameterMap(defaultParameterMap)
#parameterMapVector.append(defaultParameterMap)
#elastixImageFilter.SetParameterMap(parameterMapVector)
#elastixImageFilter.SetParameterMap(parameterMapVector)
#elastixImageFilter.SetOutputDirectory(elastix_output_dir)
#elastixImageFilter.Execute()

(192, 160, 256)
ParameterObject (0x318ac40)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 3997
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (AutomaticParameterEstimation "true")
  (CheckNumberOfSamples "true")
  (DefaultPixelValue 0)
  (FinalBSplineInterpolationOrder 3)
  (FinalGridSpacingInPhysicalUnits 8)
  (FixedImagePyramid "FixedSmoothingImagePyramid")
  (GridSpacingSchedule 2.80322 1.9881 1.41 1)
  (ImageSampler "RandomCoordinate")
  (Interpolator "LinearInterpolator")
  (MaximumNumberOfIterations 256)
  (MaximumNumberOfSamplingAttempts 8)
  (Metric "AdvancedMattesMutualInformation" "TransformBendingEnergyPenalty")
  (Metric0Weight 1)
  (Metric1Weight 1)
  (MovingImagePyramid "MovingSmoothingImagePyramid")
  (NewSamplesEveryIteration "true")
  (NumberOfResolutions 4)
  (NumberOfSamplesForExactGradient 4096)
  (NumberOfSpatialSamples 2048)
  (Optimizer "AdaptiveStochasticGradientDescent")
  (Registration "MultiMetricMul

In [ ]:
# VoxelMorph Directory
elx_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/elastix_Abd/'
elx_defGrid_dir  = elx_directory + 'defGrid_figures/'
image_paths   = get_file_paths(elx_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')

In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')

#### ANTs

In [ ]:
# ANTs Directory
ant_directory    = '/data/groups/beets-tan/l.estacio/Med_Align_Net/ants_Abd/'
ant_defGrid_dir  = ant_directory + 'defGrid_figures/'
image_paths   = get_file_paths(ant_directory)
img2_paths    = image_paths['img2.nii.gz']
seg2_paths    = image_paths['seg2.nii.gz']
w_img_paths   = image_paths['w_img.nii.gz']
w_seg_paths   = image_paths['w_seg.nii.gz']
flow_4d_paths = image_paths['flow_4d.nii.gz']

img2_0_paths    = img2_paths[0]
seg2_0_paths    = seg2_paths[0]
w_img_0_paths   = w_img_paths[0]
w_seg_0_paths   = w_seg_paths[0]
flow_4d_0_paths = flow_4d_paths[0]

img2_54_paths    = img2_paths[54]
seg2_54_paths    = seg2_paths[54]
w_img_54_paths   = w_img_paths[54]
w_seg_54_paths   = w_seg_paths[54]
flow_4d_54_paths = flow_4d_paths[54]
main(img2_0_paths, seg2_0_paths, w_img_0_paths, w_seg_0_paths, flow_4d_0_paths, 'warped_image_with_grid_0.nii.gz')

In [ ]:
main(img2_54_paths, seg2_54_paths, w_img_54_paths, w_seg_54_paths, flow_4d_54_paths, 'warped_image_with_grid_54.nii.gz')